# Kaggle team name: vijf5!

# Members:
1. Jesse Coenraad -> JesseCoenraad
2. Jordan van Sintanneland -> JordanvanSint
3. Casper Priem -> CasperPriem
4. Jassar Hasiba -> UrClades

### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\jassa\AppData\Local\Temp\ipykernel_6456\555797462.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Data inladen

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# 1. Exploratieve Data Analyse

## 1.2 Toon de datatypes en basisstatistieken van iedere kolom

In [4]:
display(train.head())
display(train.info())
display(train.describe())

,date_hour,holiday,weathersit,temp,atemp,hum,windspeed,cnt
0,2011-01-01 00:00:00,0,1,0.24,0.2879,0.81,0.0,16
1,2011-01-01 01:00:00,0,1,0.22,0.2727,0.80,0.0,40
2,2011-01-01 02:00:00,0,1,0.22,0.2727,0.80,0.0,32
3,2011-01-01 03:00:00,0,1,0.24,0.2879,0.75,0.0,13
4,2011-01-01 04:00:00,0,1,0.24,0.2879,0.75,0.0,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16637 entries, 0 to 16636
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date_hour   16637 non-null  object 
 1   holiday     16637 non-null  int64  
 2   weathersit  16637 non-null  int64  
 3   temp        16637 non-null  float64
 4   atemp       16637 non-null  float64
 5   hum         16637 non-null  float64
 6   windspeed   16637 non-null  float64
 7   cnt         16637 non-null  int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 1.0+ MB


None

,holiday,weathersit,temp,atemp,hum,windspeed,cnt
count,16637.000000,16637.000000,16637.000000,16637.000000,16637.000000,16637.000000,16637.000000
mean,0.028671,1.415580,0.504745,0.482608,0.624756,0.190310,190.477009
std,0.166885,0.637298,0.192369,0.171557,0.193227,0.121915,182.026755
min,0.000000,1.000000,0.020000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,1.000000,0.340000,0.333300,0.470000,0.104500,41.000000
50%,0.000000,1.000000,0.520000,0.500000,0.620000,0.194000,143.000000
75%,0.000000,2.000000,0.660000,0.621200,0.780000,0.253700,282.000000
max,1.000000,4.000000,1.000000,1.000000,1.000000,0.850700,977.000000


### 1.2.1 Valt hier iets op?

### 1.2.2 Beschrijf de kolommen

- date_hour: Je hebt informatie over de periode van 1-1-2011 t/m 30-11-2012, metingen per uur
- holiday: Vakantiedag of geen vakantiedag
- weathersit: Weersituatie:
    1. Helder, licht bewolkt, deels bewolkt
    2. Mistig , mistig en licht bewolkt
    3. Lichte sneeuw, lichte regen, lichte regen en onweer, zwaar bewolkt, lichte regen en zwaar bewolkt
    4. Zware regen,  hagel,  zware mist, sneeuw
- temp: genormaliseerde temperatuur
- atemp: genormaliseerde gevoelstemperatuur
- hum: genormaliseerde luchtvochtigheid
- windspeed: genormaliseerde windsnelheid
- cnt: het aantal dat per uur is verhuurd

## 1.3 Voeg tijdserie elementen toe en gebruik passende visualistaites om relevante patronen zichtbaar te maken.

*Denk hier bijvoorbeeld ook aan statistische kenmerken, zoals gemiddelde, variantie en autocorrelatie (ACF en PACF plots)

### 1.3.1 Valt hier iets op?

## 1.4* Stationariteit controleren: Stationariteit is een voorwaarde voor veel time series modellen (zoals ARIMA). Controleer op stationariteit met visuele inspectie of met statistische tests zoals de Augmented Dickey-Fuller (ADF) test.

### 1.4.1 Leg uit hoe het werkt en geef je bevindingen

## 1.5 Beschrijf wat de belangrijkste bevindingen zijn van de EDA

# 2. Data-voorbereiding

## 2.1 Verwijder indien nodig niet relevante kolommen

### 2.1.1 Leg uit waarom deze wel of niet verwijderd zijn

## 2.2 Vul eventuele missende waarden in of verwijder ze. Voor time series kan interpolatie een nuttige methode zijn.

### 2.2.1 Leg uit waarom er voor wat gekozen is

## 2.3 Kijk of er outliers zijn

### 2.3.1 Verwijder of corrigeer deze indien nodig

### 2.3.2 Leg uit waarom deze keuze is gemaakt

## 2.4 Schaal de data indien nodig

### 2.4.1 Leg uit waarom voor deze methode is gekozen en voor welke modellen dit uitmaakt

## 2.5 Data sampling indien nodig

### 2.5.1 Leg uit waarom dit wel/niet nodig is en welke methode wordt gebruikt

# 3. Time series feature engineering

Deze opdracht hangt sterk samen met de opdracht 3 (modelleren). Je zult na het toevoegen van features moeten gaan modelleren om te achterhalen wat de invloed van de features is. Daarna zul je waarschijnlijk weer teruggaan naar deze opdracht om nieuwe features uit te proberen. Dit is een iteratief proces. Zorg ervoor dat je uitwerking duidelijk maakt hoe jullie tot de uiteindelijke keuze voor de features en model(len) zijn gekomen

## 3.1 Voeg tijdserie features toe aan de dataset

## 3.2 Onderzoek van de invloed is van deze features, samen met de andere features, op de voorspellingen

### 3.2.1 Onderzoek naar seizoenspatronen met Fourier analyse

#### 3.2.2 Leg uit hoe het werkt en wat er is gevonden

### 3.2.3 Onderzoek naar trends

#### 3.2.4 Leg uit hoe het werkt en wat er is gevonden

### 3.2.5 Onderzoek naar cycli en autocorrelatie

#### 3.2.6 Leg uit hoe het werkt en wat er is gevonden

### 3.2.7 Onderzoek naar andere tijdsgebaseerde features

*denk hier bijvoorbeeld aan:

- lags: Vertragingen in de tijd (bijv. de waarde van vorige dag).
- Rolling statistics: Gemiddelden of varianties over een bepaald venster.
- Datumgerelateerde features: Dag van de week, maand, seizoenen, feestdagen, enz.

#### 3.2.8 Leg uit hoe het werkt en wat er is gevonden

## 3.3 Maak duidelijk hoe jullie tot de uiteindelijke keuze voor de features en model(len) zijn gekomen. Wat is er allemaal geprobeerd?

# 4. Train-test splitsing en validatie

*Cross-validation voor time series: Bij time series data kun je geen willekeurige splitsing gebruiken. Gebruik TimeSeriesSplit uit scikit-learn om met opeenvolgende perioden te valideren, of werk met 'rolling' en 'expanding' vensters.

## 4.1 splits de data

## 4.2 Cross-validation voor time series

### 4.2.1 Leg uit waarom dit anders is voor time series

# 5. Modelleren

# **Hou bij welke hyperparameters je gebruikt voor elke kaggle submission voor 6.3**

## 5.1 Model 1 waar je time series combineert met reguliere feautures (Lineaire regressie)?

### 5.1.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.1.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.1.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.2 Model 2 waar je time series combineert met reguliere feautures (Ensemble)?

### 5.2.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.2.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.2.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.3 Model 3 waar je time series combineert met reguliere feautures?

### 5.3.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.3.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.3.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.4 Model 4 waar je time series combineert met reguliere feautures

### 5.4.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.4.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.4.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.5 Model 5 waar je time series combineert met reguliere feautures

### 5.5.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.5.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.5.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.6 Model 1 van specifieke tijdseriemodellen (SARIMA(X))

### 5.6.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.6.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.6.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.7 Model 2 van specifieke tijdseriemodellen

### 5.7.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.7.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.7.3 Pas het model toe op de testset en upload het resultaat op Kaggle

## 5.8 Model 1 van hybdride modellen

### 5.8.1 Vind de beste hyperparameters, gebruik hier eventueel gridsearch en pipelines

### 5.8.2 Geef toelichting in de werking van het model en de gekozen parameters

### 5.8.3 Pas het model toe op de testset en upload het resultaat op Kaggle

# 6. Bevindingen

## 6.1 Geef een beknopt overzicht van de resultaten

## 6.2 Welke model presteert het beste? Welke parameters gebruikte deze?

## 6.3 Toon je scores op Kaggle en laat zien wat de resultaten waren van verbeteringen op je score op Kaggle

# 7. Conclusie en aanbevelingen

## 7.1 welke features hadden uiteindelijk het meeste invloed op de voorspelling?

## 7.2 Sluit dit aan op de EDA?

## 7.3 Welk advies kun je op basis hiervan geven aan de verhuurder?

## 7.4 Welk model zou je adviseren aan de verhuurder? Waarom? Is dit per definitie het model met de beste voorspellingen of zijn  er ook andere overwegingen? Zo ja, welke?